In [48]:
%%HTML
<button onclick="$('.output_stderr').toggle();">Toggle Code</button>

gensim中，主题模型通过Transformation接口来实现，表示将原文本转换为各种空间向量模型。

在该示例中，演示如何将文档从一种向量表达转成另一种。两个目标：
1、呈现corpus的隐含结构，发现单词之间的关系，用更语义化的方式来描述文档。比如TFIDF模型
2、使文档表达更紧凑，可以减少资源消耗，也可以通过忽略边缘化的数据（marginal data trends are ignored）而降噪。

In [38]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import tempfile
import os.path
TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

Folder "/tmp" will be used to save temporary dictionary and corpus.


## TFIDF模型

**将文本转成词袋模型，去掉停用词和只出现一次的词；将文本的每个词作为特征，计算所有特征的tfidf作为权重。再通过余弦相似度衡量文档间的相似性**

首先加载corpus模型

In [39]:
from gensim import corpora, models, similarities

if os.path.isfile(os.path.join(TEMP_FOLDER, 'deerwester.dict')):
    dictionary = corpora.Dictionary.load(os.path.join(TEMP_FOLDER, 'deerwester.dict'))
    corpus = corpora.MmCorpus(os.path.join(TEMP_FOLDER, 'deerwester.mm'))
    print("done")
else:
    print("error")

2017-11-07 11:41:00,022 : INFO : loading Dictionary object from /tmp/deerwester.dict
2017-11-07 11:41:00,024 : INFO : loaded /tmp/deerwester.dict
2017-11-07 11:41:00,026 : INFO : loaded corpus index from /tmp/deerwester.mm.index
2017-11-07 11:41:00,027 : INFO : initializing corpus reader from /tmp/deerwester.mm
2017-11-07 11:41:00,029 : INFO : accepted corpus with 9 documents, 12 features, 28 non-zero entries


done


In [40]:
print(dictionary[0])

interface


创建一个tfidf模型


In [61]:
#做归一化处理
tfidf = models.TfidfModel(corpus, normalize=True)
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

2017-11-07 16:46:11,865 : INFO : collecting document frequencies
2017-11-07 16:46:11,868 : INFO : PROGRESS: processing document #0
2017-11-07 16:46:11,870 : INFO : calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(1, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.44424552527467476), (6, 0.3244870206138555), (7, 0.3244870206138555)]
[(0, 0.5710059809418182), (6, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(2, 0.49182558987264147), (6, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (4, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(5, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


不同的Transformation模型需要不同的初始化参数，像Tfidf只需要遍历corpus就可以计算词频和idf。

假设有篇文档是如下的格式，可以得出tfidf的值

In [42]:
doc_bow = [(0, 1), (1, 1)]
#注意这里是方括号
print(tfidf[doc_bow])

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


将所有文档都转为tfidf，建立索引

In [43]:
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=12)

2017-11-07 11:41:00,079 : INFO : creating sparse index
2017-11-07 11:41:00,086 : INFO : creating sparse matrix from corpus
2017-11-07 11:41:00,088 : INFO : PROGRESS: at document #0
2017-11-07 11:41:00,091 : INFO : created <9x12 sparse matrix of type '<type 'numpy.float32'>'
	with 28 stored elements in Compressed Sparse Row format>


> 注意：similarities.MatrixSimilarity类仅仅适合能将所有的向量都在内存中的情况。例如，如果一个百万文档级的语料库使用该类，可能需要2G内存与256维LSI空间。 
如果没有足够的内存，你可以使用similarities.Similarity类。该类的操作只需要固定大小的内存，因为他将索引切分为多个文件（称为碎片）存储到硬盘上了。它实际上使用了similarities.MatrixSimilarity和similarities.SparseMatrixSimilarity两个类，因此它也是比较快的，虽然看起来更加复杂了。

索引可以通过save和load保存到本地

In [54]:
index.save(os.path.join(TEMP_FOLDER, 'deerwester_tfidf.index'))
index = similarities.SparseMatrixSimilarity.load(os.path.join(TEMP_FOLDER, 'deerwester_tfidf.index'))

2017-11-07 15:19:28,061 : INFO : saving SparseMatrixSimilarity object under /tmp/deerwester_tfidf.index, separately None
2017-11-07 15:19:28,064 : INFO : saved /tmp/deerwester_tfidf.index
2017-11-07 15:19:28,066 : INFO : loading SparseMatrixSimilarity object from /tmp/deerwester_tfidf.index
2017-11-07 15:19:28,067 : INFO : loaded /tmp/deerwester_tfidf.index


In [44]:
sims = index[tfidf[doc_bow]]
print(list(enumerate(sims)))

[(0, 0.81649655), (1, 0.31412902), (2, 0.40376222), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0)]


这里表示doc_bow和第一篇文档的相似度是0.81，第二篇是0.31，==

## LSA/LSI模型

tfidf的缺点是无法识别单词的多义性（polysems）和同义性（synonymys）。
LSA用SVD对单词-文档矩阵进行分解。应该是跟VSM相反的结构，行代表词语，列代表文档。矩阵的每个元素是词在某篇文档中的tfidf值。

SVD将维的目的：
1、原矩阵太大，减少资源消耗。
2、去噪
3、原矩阵过于稀疏。原矩阵是精确反映每个词是否出现在某篇文档，然而我们对某篇文档“相关”的所有词更感兴趣。

降维的结果是不同的词或因为其语义的相关性导致合并，如：

{(car), (truck), (flower)} --> {(1.3452 * car + 0.2828 * truck), (flower)}

降维可以解决一部分同义词的问题，也能解决一部分二义性问题。具体来说，原始词-文档矩阵经过降维处理后，原有词向量对应的二义部分会加到和其语义相似的词上，而剩余部分则减少对应的二义分量。

缺点是新生成的矩阵解释性比较差，因为SVD只是一种数学变换，无法对应现实中的概念。
- LSA无法扑捉一词多以的现象。在原始词-向量矩阵中，每个文档的每个词只能有一个含义。比如同一篇文章中的“The Chair of Board"和"the chair maker"的chair会被认为一样。在语义空间中，含有一词多意现象的词其向量会呈现多个语义的平均。相应的，如果有其中一个含义出现的特别频繁，则语义向量会向其倾斜。
- LSA具有词袋模型的缺点，即在一篇文章，或者一个句子中忽略词语的先后顺序。
- LSA的概率模型假设文档和词的分布是服从联合正态分布的，但从观测数据来看是服从泊松分布的。因此LSA算法的一个改进PLSA使用了多项分布，其效果要好于LSA。


In [50]:
#   num_topics=2，也就是通过SVD降维成2维。实际中，200-500是推荐维度。
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)
#在最初的corpus上包装了两层： bow->tfidf->fold-in-lsi
corpus_lsi = lsi[corpus_tfidf]
#看每个topic的最相关单词
lsi.print_topics(2)

2017-11-07 11:53:58,774 : INFO : using serial LSI version on this node
2017-11-07 11:53:58,783 : INFO : updating model with new documents
2017-11-07 11:53:58,786 : INFO : preparing a new chunk of documents
2017-11-07 11:53:58,789 : INFO : using 100 extra samples and 2 power iterations
2017-11-07 11:53:58,792 : INFO : 1st phase: constructing (12, 102) action matrix
2017-11-07 11:53:58,806 : INFO : orthonormalizing (12, 102) action matrix
2017-11-07 11:53:58,810 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2017-11-07 11:53:58,815 : INFO : computing the final decomposition
2017-11-07 11:53:58,821 : INFO : keeping 2 factors (discarding 47.565% of energy spectrum)
2017-11-07 11:53:58,825 : INFO : processed documents up to #9
2017-11-07 11:53:58,827 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2017-11-07 11:53:58,829 : INFO : topic #

[(0,
  u'0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  u'-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"time" + -0.320*"response" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"')]

能看到通过LSA，trees，graph，minors都是相关的词，也对第一个主题贡献最多。

In [57]:
for doc in corpus_lsi:
    print(doc)

[(0, 0.066007833960901874), (1, -0.52007033063618535)]
[(0, 0.19667592859142313), (1, -0.76095631677000408)]
[(0, 0.089926399724462813), (1, -0.7241860626752511)]
[(0, 0.075858476521780266), (1, -0.63205515860034323)]
[(0, 0.10150299184979986), (1, -0.57373084830029497)]
[(0, 0.70321089393783098), (1, 0.16115180214025623)]
[(0, 0.87747876731198315), (1, 0.16758906864659218)]
[(0, 0.90986246868185794), (1, 0.14086553628718843)]
[(0, 0.61658253505692806), (1, -0.053929075663894405)]


可以看到前5篇文档与第二个topic相关，后面几个与第一个topic相关（**负的也能表示相关**？）

In [58]:
lsi.save(os.path.join(TEMP_FOLDER, 'model.lsi'))

2017-11-07 16:42:00,192 : INFO : saving Projection object under /tmp/model.lsi.projection, separately None
2017-11-07 16:42:00,194 : INFO : saved /tmp/model.lsi.projection
2017-11-07 16:42:00,196 : INFO : saving LsiModel object under /tmp/model.lsi, separately None
2017-11-07 16:42:00,199 : INFO : not storing attribute projection
2017-11-07 16:42:00,201 : INFO : not storing attribute dispatcher
2017-11-07 16:42:00,202 : INFO : saved /tmp/model.lsi


## 其他常用模型

gensim实现了几种常用的空间向量模型算法

### TFIDF
略

### LSA

In [63]:
model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=300)

2017-11-07 17:30:10,967 : INFO : using serial LSI version on this node
2017-11-07 17:30:10,968 : INFO : updating model with new documents
2017-11-07 17:30:10,971 : INFO : preparing a new chunk of documents
2017-11-07 17:30:10,972 : INFO : using 100 extra samples and 2 power iterations
2017-11-07 17:30:10,973 : INFO : 1st phase: constructing (12, 400) action matrix
2017-11-07 17:30:10,975 : INFO : orthonormalizing (12, 400) action matrix
2017-11-07 17:30:10,977 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2017-11-07 17:30:10,978 : INFO : computing the final decomposition
2017-11-07 17:30:10,980 : INFO : keeping 9 factors (discarding 0.000% of energy spectrum)
2017-11-07 17:30:10,983 : INFO : processed documents up to #9
2017-11-07 17:30:10,984 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2017-11-07 17:30:10,985 : INFO : topic #1

LSI可以实现在线训练，即可以在训练时不断的喂新的文档。
> 比如：
```
#增加新文档进行训练
model.add_documents(another_tfidf_corpus)
#把新的文档映射到LSI空间，而不去影响LSI模型
lsi_vec = model[tfidf_vec]
```

在[gensim.models.lsimodel](https://radimrehurek.com/gensim/models/lsimodel.html#module-gensim.models.lsimodel)可以看到如果使LSI在不断增加新文档的时候逐渐遗忘旧的训练集。也可以进行分布式训练。

### Random Projections
略

### LDA

In [64]:
model = models.LdaModel(corpus, id2word=dictionary, num_topics=100)

2017-11-07 17:38:31,466 : INFO : using symmetric alpha at 0.01
2017-11-07 17:38:31,469 : INFO : using symmetric eta at 0.0833333333333
2017-11-07 17:38:31,470 : INFO : using serial LDA version on this node
2017-11-07 17:38:31,494 : INFO : running online LDA training, 100 topics, 1 passes over the supplied corpus of 9 documents, updating model once every 9 documents, evaluating perplexity every 9 documents, iterating 50x with a convergence threshold of 0.001000
2017-11-07 17:38:31,495 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2017-11-07 17:38:31,539 : INFO : -124.846 per-word bound, 38225949658049039314110059938796535808.0 perplexity estimate based on a held-out corpus of 9 documents with 29 words
2017-11-07 17:38:31,540 : INFO : PROGRESS: pass 0, at document #9/9
2017-11-07 17:38:31,561 : INFO : topic #27 (0.010): 0.083*"user" + 0.083*"survey" + 0.083*"graph" + 0.083*"trees" + 0.083*"eps" + 0.083

### HDP

In [65]:
model = models.HdpModel(corpus, id2word=dictionary)

2017-11-07 17:48:10,550 : INFO : (0, u'0.446*trees + 0.103*computer + 0.088*interface + 0.088*time + 0.066*eps + 0.056*graph + 0.051*survey + 0.044*human + 0.024*system + 0.015*response')
2017-11-07 17:48:10,551 : INFO : (1, u'0.241*graph + 0.148*user + 0.122*interface + 0.106*time + 0.094*human + 0.085*response + 0.071*computer + 0.069*minors + 0.030*survey + 0.015*system')
2017-11-07 17:48:10,551 : INFO : (2, u'0.244*human + 0.183*time + 0.136*computer + 0.123*survey + 0.093*response + 0.057*user + 0.048*graph + 0.042*system + 0.028*eps + 0.024*trees')
2017-11-07 17:48:10,552 : INFO : (3, u'0.219*response + 0.216*human + 0.184*computer + 0.090*system + 0.073*graph + 0.050*eps + 0.044*time + 0.033*user + 0.030*trees + 0.025*interface')
2017-11-07 17:48:10,553 : INFO : (4, u'0.252*interface + 0.182*human + 0.137*time + 0.107*eps + 0.102*graph + 0.064*minors + 0.063*survey + 0.038*user + 0.036*trees + 0.016*response')
2017-11-07 17:48:10,554 : INFO : (5, u'0.213*human + 0.142*computer +